The Grand Hideaway Zone inn
-----------------------------
Challenge statement
-----------------------------
The Grand Hideaway Zone is an inn which prides itself in the privacy it offers to its clients. It boasts private dining rooms and a secret meeting lounge in the basement. It even accepts IDs that are known to be fake. It is no surprise that it tends to attract all kinds of suspicious characters, so watch out if you're staying here!

In one of its latest efforts to give even more privacy to its clients, it now offers quantum communication services. It means that its telephone and internet lines use quantum communication protocols. Messages can no longer be eavesdropped on, to the police's dismay.

However, the communication protocols rely on Greenberger–Horne–Zeilinger states, or GHZ states for short, which are entangled qubit states of the form
$\left| \text{GHZ} \right\rangle = \frac{1}{\sqrt{2}} \left| 00\ldots0 \right\rangle + \frac{1}{\sqrt{2}} \left| 11\ldots1 \right\rangle$

Generating these states uses two-qubit gates, which can be noisy. While the inn uses a state-of-the-art neutral-atom quantum computer to generate them, the communication is not foolproof and several misunderstandings have been known to happen, one of which led to the capture of a mafia boss.

In this challenge, we have access to the inn's neutral-atom quantum computer. Using laser pulses and carefully controlling the weak van der Waals interactions between atoms, this device can implement the single qubit rotations RX and RY, and the CZ (Control-Z) gate.
Your task is to generate a GHZ state for a given number of qubits $n_qubits$ using only rotations and the CZ gate. However, as is the case with most real-life quantum devices, two-qubit gates are a source of noise. We model the noise by adding a depolarizing gate on the target qubit after each CZ gate, with noise parameterγ. Therefore, the GHZ state that the neutral-atom device generates will be noisy. You must quantify how good the produced GHZ state is by calculating the fidelity between the noisy state and the ideal GHZ state.

You will now be able to tell how error-prone the inn's messaging services actually are!

Challenge Code
----------------
In the challenge template, you must complete the following functions.

* 'GHZ_circuit': A quantum function that, given the noise parameter ('noise_param' ('float')) γ and the number of qubits 'n_qubits' ('int'), generates the GHZ state in 'n_qubits' dimensions. This circuit may only use the RX (qml.RX), RY (qml.RY), and CZ (qml.CZ) gates and the depolarizing channel (qml.DepolarizingChannel).

* 'GHZ_fidelity': Given γ ('noise_param' ('float')) and the number of qubits 'n_qubits' ('int'), calculates the fidelity between the GHZ state generated by GHZ_circuit and the ideal

You are also given some space to write some helper functions.

Input
----------

As an input to this challenge, you are given γ (float) and $n_qubits$ ('int') in this order.

Output
-------------

The expected output is the fidelity (float) as calculated by the GHZ_fidelity function.

In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np

# Write any helper functions you need here


def GHZ_circuit(noise_param, n_qubits):

    """
    Quantum circuit that prepares an imperfect GHZ state using gates native to a neutral atom device.

    Args:
        - noise_param (float): Parameter that quantifies the noise in the CZ gate, modelled as a
        depolarizing channel on the target qubit. noise_param is the parameter of the depolarizing channel
        following the PennyLane convention.
        - n_qubits (int): The number of qubits in the prepared GHZ state.
    Returns:
        - (np.tensor): A density matrix, as returned by `qml.state`, representing the imperfect GHZ state.

    """



    # Put your code here



def GHZ_fidelity(noise_param, n_qubits):

    """
    Calculates the fidelity between the imperfect GHZ state returned by GHZ_circuit and the ideal GHZ state.

    Args:
        - noise_param (float): Parameter that quantifies the noise in the CZ gate, modelled as a
        depolarizing channel on the target qubit. noise_param is the parameter of the depolarizing channel
        following the PennyLane convention.
        - n_qubits (int): The number of qubits in the GHZ state.
    Returns:
        - (float): The fidelity between the noisy and ideal GHZ states.
    """

    dev = qml.device('default.mixed', wires=n_qubits)

    GHZ_QNode = qml.QNode(GHZ_circuit,dev)



    # Use GHZ_QNode to find the fidelity between
    # the noisy GHZ state and an ideal GHZ state



# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    output = GHZ_fidelity(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    dev = qml.device('default.mixed', wires=4)
    qnode = qml.QNode(GHZ_circuit, dev)
    u = qnode(0.05,3)

    for op in qnode.tape.operations:
        assert (isinstance(op, qml.RX) or isinstance(op, qml.RY) or isinstance(op, qml.CZ) or isinstance(op, qml.DepolarizingChannel)), "You are using forbidden gates!"

    assert np.isclose(solution_output, expected_output, rtol = 1e-4)


# These are the public test cases
test_cases = [
    ('[0.05, 3]', '0.9027779255467782'),
    ('[0.01, 5]', '0.9606614879634601')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

GHZ state using qml.Hadamard and qml.CNOT

In [2]:
import pennylane as qml

def generate_ghz_state(n_qubits):
    """
    Function to generate a GHZ state for a given number of qubits.

    Args:
        n_qubits (int): The number of qubits in the GHZ state.

    Returns:
        Function that prepares the GHZ state when called.
    """

    # Define a quantum device with the specified number of qubits
    dev = qml.device('default.qubit', wires=n_qubits)

    # Define the quantum function to prepare the GHZ state
    @qml.qnode(dev)
    def prepare_ghz_state():
        # Create a superposition of the first qubit
        qml.Hadamard(wires=0)

        # Entangle the first qubit with each of the other qubits
        for i in range(1, n_qubits):
            qml.CNOT(wires=[0, i])

        # Measurement (optional, just to inspect the state)
        return qml.state()

    return prepare_ghz_state

# Example usage:
n_qubits = 4  # Specify the number of qubits for the GHZ state
ghz_state_function = generate_ghz_state(n_qubits)
ghz_state = ghz_state_function()
print(ghz_state)


[0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [3]:
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Statevector
state = Statevector(ghz_state)
state.draw('latex')

<IPython.core.display.Latex object>

Function to generate a GHZ state for a given number of qubits using only RY, RX, CZ

In [10]:
import pennylane as qml
import pennylane.numpy as np

def generate_ghz_state(n_qubits):
    """
    Function to generate a GHZ state for a given number of qubits.

    Args:
        n_qubits (int): The number of qubits in the GHZ state.

    Returns:
        Function that prepares the GHZ state when called.
    """

    # Define a quantum device with the specified number of qubits
    dev = qml.device('default.qubit', wires=n_qubits)

    # Define the quantum function to prepare the GHZ state
    @qml.qnode(dev)
    def prepare_ghz_state():
        # Step 1: Initialize the first qubit in a superposition state
        qml.RY(np.pi/2, wires=0)

        # Step 2: Entangle the first qubit with each of the other qubits
        for i in range(1, n_qubits):
            # Use RX gates to transform the action of CZ to CNOT
            qml.RX(np.pi/2, wires=i)
            qml.CZ(wires=[0, i])
            qml.RX(-np.pi/2, wires=i)

        # Measurement (optional, just to inspect the state)
        return qml.state()

    return prepare_ghz_state

# Example usage:
n_qubits = 4  # Specify the number of qubits for the GHZ state
ghz_state_function = generate_ghz_state(n_qubits)
ghz_state = ghz_state_function()
print(ghz_state)

[0.70710678+0.j         0.        +0.j         0.        +0.j
 0.        +0.j         0.        +0.j         0.        +0.j
 0.        +0.j         0.        +0.j         0.        +0.j
 0.        +0.j         0.        +0.j         0.        +0.j
 0.        +0.j         0.        +0.j         0.        +0.j
 0.        -0.70710678j]


In [11]:
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Statevector
state = Statevector(ghz_state)
state.draw('latex')

<IPython.core.display.Latex object>

In [18]:
dev = qml.device('default.mixed', wires=n_qubits)
@qml.qnode(dev)
def circuit(noise_param, n_qubits):
    # Create an initial superposition of all qubits using Hadamard gate on the first qubit
    qml.RY(np.pi/2, wires=0)
    # Step 2: Entangle the first qubit with each of the other qubits
    for i in range(1, n_qubits):
        # Use RX gates to transform the action of CZ to CNOT
        qml.RX(np.pi/2, wires=i)
        qml.CZ(wires=[0, i])
        qml.DepolarizingChannel(noise_param, wires=0)
        qml.RX(-np.pi/2, wires=i)
    # Apply the depolarizing channel to each qubit
    return qml.state()

In [19]:
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Statevector
n_qubits = 4
noise_param = 0.05
ghz_state = circuit(noise_param,n_qubits)
print(ghz_state)
state = Statevector(ghz_state, n_qubits)
state.draw('latex')

[[0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0.5+0.j
  0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
  0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
  0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
  0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
  0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
  0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j
  0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. 

QiskitError: 'Invalid input: not a vector or column-vector.'

In [21]:
import pennylane as qml
from pennylane import numpy as np

def GHZ_circuit(n_qubits, noise_param):
    dev = qml.device('default.mixed', wires=n_qubits)

    @qml.qnode(dev)
    def circuit():
        qml.RY(np.pi/2, wires=0)
        for i in range(n_qubits - 1):
            qml.CZ(wires=[i, i + 1])
        for i in range(n_qubits):
            qml.DepolarizingChannel(noise_param, wires=i)
        return qml.state()

    ghz_state_noisy = circuit()
    return ghz_state_noisy
def GHZ_fidelity(n_qubits, noise_param):
    # Ideal GHZ state
    ideal_ghz_state = np.zeros(2**n_qubits)
    ideal_ghz_state[0] = ideal_ghz_state[-1] = 1 / np.sqrt(2)

    # Get the noisy GHZ state from the GHZ_circuit
    noisy_ghz_state = GHZ_circuit(n_qubits, noise_param)

    # Calculate fidelity using PennyLane's fidelity function
    fidelity = qml.math.fidelity(noisy_ghz_state, ideal_ghz_state)
    return fidelity
GHZ_fidelity(3,0.05)

LinAlgError: 1-dimensional array given. Array must be at least two-dimensional

In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np

# Write any helper functions you need here


def GHZ_circuit(noise_param, n_qubits):

    """
    Quantum circuit that prepares an imperfect GHZ state using gates native to a neutral atom device.

    Args:
        - noise_param (float): Parameter that quantifies the noise in the CZ gate, modelled as a
        depolarizing channel on the target qubit. noise_param is the parameter of the depolarizing channel
        following the PennyLane convention.
        - n_qubits (int): The number of qubits in the prepared GHZ state.
    Returns:
        - (np.tensor): A density matrix, as returned by `qml.state`, representing the imperfect GHZ state.

    """



    # Put your code here



def GHZ_fidelity(noise_param, n_qubits):

    """
    Calculates the fidelity between the imperfect GHZ state returned by GHZ_circuit and the ideal GHZ state.

    Args:
        - noise_param (float): Parameter that quantifies the noise in the CZ gate, modelled as a
        depolarizing channel on the target qubit. noise_param is the parameter of the depolarizing channel
        following the PennyLane convention.
        - n_qubits (int): The number of qubits in the GHZ state.
    Returns:
        - (float): The fidelity between the noisy and ideal GHZ states.
    """

    dev = qml.device('default.mixed', wires=n_qubits)

    GHZ_QNode = qml.QNode(GHZ_circuit,dev)



    # Use GHZ_QNode to find the fidelity between
    # the noisy GHZ state and an ideal GHZ state



# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    output = GHZ_fidelity(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    dev = qml.device('default.mixed', wires=4)
    qnode = qml.QNode(GHZ_circuit, dev)
    u = qnode(0.05,3)

    for op in qnode.tape.operations:
        assert (isinstance(op, qml.RX) or isinstance(op, qml.RY) or isinstance(op, qml.CZ) or isinstance(op, qml.DepolarizingChannel)), "You are using forbidden gates!"

    assert np.isclose(solution_output, expected_output, rtol = 1e-4)


# These are the public test cases
test_cases = [
    ('[0.05, 3]', '0.9027779255467782'),
    ('[0.01, 5]', '0.9606614879634601')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")